In [215]:
import requests
import json
import time
import copy
import re
import math
from bs4 import BeautifulSoup as bs

In [122]:
def web_request(method_name, url, dict_data, is_urlencoded=True):
    """Web GET or POST request를 호출 후 그 결과를 dict형으로 반환 """
    method_name = method_name.upper()  # 메소드이름을 대문자로 바꾼다
    if method_name not in ('GET', 'POST'):
        raise Exception('method_name is GET or POST plz...')

    if method_name == 'GET':  # GET방식인 경우
        response = requests.get(url=url, params=dict_data)
    elif method_name == 'POST':  # POST방식인 경우
        if is_urlencoded is True:
            response = requests.post(url=url, data=dict_data,
                                     headers={'Content-Type': 'application/x-www-form-urlencoded'})
        else:
            response = requests.post(url=url, data=json.dumps(dict_data), headers={'Content-Type': 'application/json'})

    dict_meta = {'status_code': response.status_code, 'ok': response.ok, 'encoding': response.encoding,
                 'Content-Type': response.headers['Content-Type']}
    if 'json' in str(response.headers['Content-Type']):  # JSON 형태인 경우
        return {**dict_meta, **response.json()}
    else:  # 문자열 형태인 경우
        return {**dict_meta, **{'text': response.text}}

In [190]:
body = [('q_hiddenVal', 1),
 ('q_searchAt', 'Y'),
 ('q_moreSvc', 1001),
 ('q_tabVal', 1001),
 ('q_svcClCode', 1001),
 ('q_atdrcCode', 11710)]

In [146]:
response = web_request('POST', 'https://icare.seoul.go.kr/icare/user/fcltyInfoManage/BD_selectFcltyInfoManageList.do', body)

In [147]:
res_html = bs(response['text'], "html.parser")

In [182]:
# q_searchVal = 키움센터명 검색시
# q_moreSvc = q_svcClCode, q_clturEvent = q_tabVal
코드값목록 = []
전체조회목록 = ['q_fcltyCl','q_ageSeCode', 'q_atdrcCode', 'q_svcClCode']
키목록 = []

for item in res_html.select('table.center tbody li'):
    if item.select_one('input') != None:
        키목록.append(item.select_one('input')['name'])
        코드값목록.append({'key':item.select_one('input')['name'], 'value':item.select_one('input')['value'], 'label':item.text})

for item in res_html.select('div.mt_10 li'):
    코드값목록.append({'key':'q_tabVal'
                  , 'value':item.select_one('a')['onclick'][item.select_one('a')['onclick'].index('(')+2:item.select_one('a')['onclick'].index(')')-1]
                  , 'label':item.text})

키목록 = list(set(키목록))
키목록.append('q_tabVal')
키목록.append('q_moreSvc')

In [191]:
body = [
    ('q_hiddenVal', '1'), 
    ('q_searchAt', 'Y'),
]
for item in 코드값목록:
    if item['key'] in 전체조회목록:
        body.append((item['key'], item['value']))

In [186]:
for item in 코드값목록:
    if item['key'] in ['q_svcClCode','q_clturEvent', 'q_tabVal']:
        print(item)

{'key': 'q_svcClCode', 'value': '1001', 'label': '우리동네 키움센터'}
{'key': 'q_svcClCode', 'value': '1003', 'label': '지역 아동센터'}
{'key': 'q_svcClCode', 'value': '1005', 'label': '초등 돌봄교실'}
{'key': 'q_svcClCode', 'value': '1002', 'label': '공동육아방'}
{'key': 'q_svcClCode', 'value': '1004', 'label': '공동육아 나눔터'}
{'key': 'q_svcClCode', 'value': '1010', 'label': '청소년방과후아카데미'}
{'key': 'q_svcClCode', 'value': '1006', 'label': '영·유아 보육시설'}
{'key': 'q_clturEvent', 'value': '2002', 'label': '초등생 교육 프로그램'}
{'key': 'q_clturEvent', 'value': '2003', 'label': '영·유아 교육 프로그램'}
{'key': 'q_tabVal', 'value': '', 'label': '전체'}
{'key': 'q_tabVal', 'value': '1001', 'label': '우리동네 키움센터'}
{'key': 'q_tabVal', 'value': '1003', 'label': '지역 아동센터'}
{'key': 'q_tabVal', 'value': '1005', 'label': '초등돌봄교실'}
{'key': 'q_tabVal', 'value': '1002', 'label': '공동육아방'}
{'key': 'q_tabVal', 'value': '1004', 'label': '공동육아 나눔터'}
{'key': 'q_tabVal', 'value': '1010', 'label': '방과후 아카데미'}
{'key': 'q_tabVal', 'value': '1006', 'label': '영.유아 

In [192]:
## 페이지당 카운트는 15개로 고정되어 있음
## 서비스별로 루프를 돌아서 처리해야함..
body.extend([('q_moreSvc', '1001'), ('q_tabVal', '1001'),
             ('q_rowPerPage', '50'),('q_currPage','1')])

In [193]:
response = web_request('POST', 'https://icare.seoul.go.kr/icare/user/fcltyInfoManage/BD_selectFcltyInfoManageList.do', body)

In [194]:
res_html = bs(response['text'], "html.parser")

In [216]:
total_cnt = res_html.select_one('p.board_info').text[res_html.select_one('p.board_info').text.index('전체')
                                                     :res_html.select_one('p.board_info').text.index('건')]
total_cnt = re.sub(r'[^0-9]', '', total_cnt)
total_pages = math.ceil(int(total_cnt)/15)

In [271]:
for item in res_html.select('ul li'):
    if item.select_one('p.box_info_p') != None:
        print(item.select_one('a')['onclick'])
        for inside_item in item.select('span'):
            print(inside_item.text.strip().replace('\t', '').replace('\n',':'))

fnView("1009","YC230101","양천구 5호점 우리동네키움센터 (융합형)","");
양천구 5호점 우리동네키움센터 (융합형)
주소서울특별시 양천구 신월로24길 19
연락처:070-:8133-:2463
이용료::::월 / ::::50,000 원::::  ::일 / ::::2,500 원
융합형키움센터:양천구:초등학생
fnView("1001","KC230101","금천구 4호점 우리동네키움센터(시흥1동)","");
금천구 4호점 우리동네키움센터(시흥1동)
주소서울특별시 금천구 시흥대로 164
연락처:02-:809-:1999
이용료::::월 / ::::50,000 원::::  ::일 / ::::2,500 원
우리동네키움센터:금천구:초등학생
fnView("1001","GS221201","강서6호점 화곡8동 우리동네키움센터","");
강서6호점 화곡8동 우리동네키움센터
주소서울특별시 강서구 강서로5길 50
연락처:02-:2602-:1067
이용료::::월 / ::::50,000 원::::  ::일 / ::::2,500 원
우리동네키움센터:강서구:초등학생
fnView("1009","GB221101","강북6호점 우리동네키움센터(융합형)(삼각산동)","");
강북6호점 우리동네키움센터(융합형)(삼각산동)
주소서울특별시 강북구 삼양로19길 102
연락처:02-:988-:1318
이용료::::월 / ::::50,000 원::::  ::일 / ::::2,500 원
융합형키움센터:강북구:초등학생
fnView("1001","DM221001","동대문구 6호점 우리동네키움센터","");
동대문구 6호점 우리동네키움센터
주소서울특별시 동대문구 약령시로5길 22
연락처:02-:920-:4541
이용료::::월 / ::::50,000 원::::  ::일 / ::::2,500 원
우리동네키움센터:동대문구:초등학생
fnView("1001","GA220501","관악3호점 우리동네키움센터","");
관악3호점 우리동네키움센터
주소서울특별시 관악구 청림2길 37
연락처:02-:877

In [ ]:
var fnView = function(svcClCode, fcltyId, fcltyNm, siteUrl) {
    if (svcClCode == "1005" || svcClCode == "2003") {
        window.open(siteUrl, "_blank");
    } else if (svcClCode == "1007" || svcClCode == "2003") {
        $("#q_fclty").val(svcClCode);
        $("#q_clturEventEtcSn").val(fcltyId);
        $("#searchForm").attr("action", "BD_selectFcltyInfoManage.do");
        $("#searchForm").submit();
    } else if (svcClCode == "2001") {
        $("#q_fclty").val(svcClCode);
        $("#q_clturEventEtcSn").val(fcltyId);
        $("#q_gubun").val("1001");
        $("#searchForm").attr("action", "BD_selectFcltyInfoManage.do");
        $("#searchForm").submit();
    } else if (svcClCode == "2002") {
        $("#q_fclty").val(svcClCode);
        $("#q_clturEventEtcSn").val(fcltyId);
        $("#q_gubun").val("1002");
        $("#searchForm").attr("action", "BD_selectFcltyInfoManage.do");
        $("#searchForm").submit();
    } else if (svcClCode == "1006") {
        if (fcltyNm.indexOf("휴일보육") > 0) {
            window.open("https://iseoul.seoul.go.kr/portal/reservation/holi_info.do", "_blank");
        } else if (fcltyNm.indexOf("시간보육") > 0) {
            window.open("https://iseoul.seoul.go.kr/portal/reservation/night_info.do", "_blank");
        } else if (fcltyNm.indexOf("365열린어린이집") > 0) {
            window.open("https://iseoul.seoul.go.kr/portal/reservation/holi_info.do", "_blank");
        } else if (fcltyNm.indexOf("거점형시간연장보육") > 0) {
            window.open("https://iseoul.seoul.go.kr/portal/reservation/dailyOpenPreschoolMiddle.do", "_blank");
        } else {
            window.open("https://iseoul.seoul.go.kr/portal/info/preSchoolList.do", "_blank");
        }
    } else if (svcClCode == "1008") {
        location.href = siteUrl;
    } else {
        $("#q_fclty").val(svcClCode);
        $("#q_fcltyId").val(fcltyId);
        $("#searchForm").attr("action", "BD_selectFcltyInfoManage.do");
        $("#searchForm").submit();
    }
}